<a href="https://colab.research.google.com/github/SebastianBentert/example_code/blob/main/bingbu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean, stdev
from itertools import combinations
import numpy as np
from scipy.stats import shapiro, mannwhitneyu, ttest_ind

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
BWUNB=[0.15,0.15,0.15]
BWU=[0.15,0.15,0.15]
BWUE=[0.15,0.15,0.15]
BWE=[0.15,0.15,0.15]
LPUNB=[0.343,0.348,0.344]
LPU=[0.347,0.349,0.349]
LPUE=[0.347,0.349,0.349]
LPE=[0.347,0.349,0.349]
C = 15
VF = C/0.5

In [ ]:
def bing(x,y):
  dall = x+y
  _, p = shapiro(dall)
  if p <= 0.05:
    _, p2 = mannwhitneyu(x, y)
    if p2 <= 0.05:
      print(f'Normalverteilung\nDaten sind unterschiedlich\nshapiro P-Wert = {round(p,3)}, U-Test P-wert = {round(p2,3)}')
    else:
      print(f'Normalverteilung\nDaten sind nicht unterschiedlich\nshapiro P-Wert = {round(p,3)}, U-Test P-wert = {round(p2,3)}')
  else:
    _, p2 = ttest_ind(x, y)
    if p2 <= 0.05:
      print(f'keine Normalverteilung\nDaten sind unterschiedlich\nshapiro P-Wert = {round(p,3)}, T-Test P-wert = {round(p2,3)}')
    else:
      print(f'kein Normalverteilung\nDaten sind nicht unterschiedlich\nshapiro P-Wert = {round(p,3)}, T-Test P-wert = {round(p2,3)}')

In [ ]:
def bing2(unb=BWUNB,u=BWU,ue=BWUE,e=BWE,vf=VF,unb2=LPUNB,u2=LPU,ue2=LPUE,e2=LPE,c=C):
  df = pd.DataFrame(data=[[unb,unb2,mean(unb),stdev(unb),mean(unb2),stdev(unb2)],[u,u2,mean(u),stdev(u),mean(u2),stdev(u2)],[ue,ue2,mean(ue),stdev(ue),mean(ue2),stdev(ue2)],[e,e2,mean(e),stdev(e),mean(e2),stdev(e2)]],
                    index=['unb','U','UE','E'],
                    columns=['Messungen Blindwert','Messungen LPl','BW mean','BW std','LPl mean','LPl std'])
  df['bw']=df['LPl mean']-df['BW mean']
  df['VF']=vf
  df['absorb']=df['bw']*df['VF']
  df['protein g/L'] = (df['bw']-0.0068)/0.054
  df['Protein mit VF g/L'] = df['protein g/L']*df['VF']
  df['Protein %'] = df['Protein mit VF g/L']/10
  fig = plt.figure(figsize=(15,3))
  ax = df['Protein %'].plot(kind='bar',yerr=df['LPl std']*100)
  plt.title('Proteingehalt %')
  for p in ax.patches:
    ax.annotate('{:.2f}'.format(round(p.get_height(),2))+'%', (p.get_x()+0.05, p.get_height() +0.1))
  print(df.T)

  #shapiro tests
  unbshapiro = [(((np.array(df['Messungen LPl']['unb'])-np.array(df['Messungen Blindwert']['unb']))-0.0068)/0.054)*df['VF']['unb'],'unbehandelt']
  Ushapiro = [(((np.array(df['Messungen LPl']['U'])-np.array(df['Messungen Blindwert']['U']))-0.0068)/0.054)*df['VF']['U'],'U']
  UEshapiro = [(((np.array(df['Messungen LPl']['UE'])-np.array(df['Messungen Blindwert']['UE']))-0.0068)/0.054)*df['VF']['UE'],'UE']
  Eshapiro = [(((np.array(df['Messungen LPl']['E'])-np.array(df['Messungen Blindwert']['E']))-0.0068)/0.054)*df['VF']['E'],'E']
  for x,y in combinations([unbshapiro,Ushapiro,UEshapiro,Eshapiro],2):
    print(f'\nVergleich von {x[1]} und {y[1]}:')
    bing(x[0],y[0])

  df.index = df.index+' '+str(c)
  df.to_excel('test.xlsx')
  fig.savefig('pic')